In [1]:
import sys
import os
sys.path.append('../src/util/')
import numpy as np
import pandas as pd

from general import (IGNLidarProcessor, Sentinel2Reader,
                     load_dem_utm, PcdGenerator,
                     PointCloudHandler, PcdFilter)

input_file = "../src/dem-data/LHD_FXX_0982_6524_PTS_C_LAMB93_IGN69.copc.laz"
sample_fraction = 0.2

processor = IGNLidarProcessor(input_file)
processor.read_point_cloud()
processor.transform_coordinates()
processor.downsample(sample_fraction=sample_fraction)
processor.generate_color()
df_ign = processor.df
df_ign.head()

Point cloud downsampled with sample fraction 0.2


,x,y,z,classification,color
259626,315601.754841,5.069443e+06,1603.25,3,"(31, 119, 180)"
2419389,315617.506961,5.069177e+06,1677.62,5,"(255, 127, 14)"
17900936,315742.764659,5.069129e+06,1720.02,5,"(255, 127, 14)"
3666107,315680.390416,5.069146e+06,1699.57,5,"(255, 127, 14)"
20740605,315874.449597,5.069438e+06,1800.28,2,"(214, 39, 40)"


In [2]:
token = os.environ.get('hdb_token')
product_path = "/home/ubuntu/project/destine-godot-mvp/src/sentinel2-data/T32TLR_20241030T103151_TCI_20m.jp2"
reader = Sentinel2Reader(filepath=product_path, preprocess=True)
bounds = reader.bounds
width = reader.width
height = reader.height
parameter = 'dem'

dem_data = load_dem_utm(token, parameter, bounds, width, height)
# Initialize and generate point cloud
pcd_gen = PcdGenerator(sat_data=reader.data,
                       dem_data=dem_data,
                       sample_fraction=sample_fraction)
pcd_gen.generate_point_cloud()
# pcd_gen.downsample(sample_fraction=sample_fraction)
df = pcd_gen.df
df.head()

Total points before sampling: 30140100


,x,y,z,color
0,300010,4990210,2474.875,"(116, 95, 79)"
1,300030,4990210,2468.250,"(112, 91, 73)"
2,300050,4990210,2459.250,"(128, 107, 88)"
3,300070,4990210,2448.500,"(139, 113, 94)"
4,300090,4990210,2435.250,"(143, 123, 104)"


In [3]:
filterer = PcdFilter(df=df, df_target=df_ign)
filterer.set_bounding_box_from_target(margin=4)
df_filtered = filterer.filter_data()
pcd = filterer.concatenate_dataframes([df_filtered, df_ign])

In [4]:
handler = PointCloudHandler(pcd)
handler.to_open3d()
handler.save_point_cloud(filename="final_IGN_LiDAR_HD.ply")

Point cloud saved to final_IGN_LiDAR_HD.ply


In [5]:
import folium
from pyproj import Transformer

# Initialize transformer to convert UTM (EPSG:32632) to WGS84 (EPSG:4326)
transformer_to_wgs84 = Transformer.from_crs("epsg:32632", "epsg:4326", always_xy=True)

def get_bbox_coordinates(df):
    """Calculate bounding box (min/max) and convert UTM to WGS84."""
    x_min, x_max = df["x"].min(), df["x"].max()
    y_min, y_max = df["y"].min(), df["y"].max()

    lat_min, lon_min = transformer_to_wgs84.transform(x_min, y_min)
    lat_max, lon_max = transformer_to_wgs84.transform(x_max, y_max)
    return (lat_min, lon_min), (lat_max, lon_max), transformer_to_wgs84.transform((x_min + x_max) / 2, (y_min + y_max) / 2)

# Get bounding boxes and map center
bbox1_min, bbox1_max, center1 = get_bbox_coordinates(df)
bbox2_min, bbox2_max, center2 = get_bbox_coordinates(df_ign)

# Define map center as the midpoint between the two dataset centers
map_center = [(center1[1] + center2[1]) / 2, (center1[0] + center2[0]) / 2]

# Initialize Folium map
m = folium.Map(location=map_center, zoom_start=8, tiles='OpenStreetMap')

# Add first bounding box (df)
folium.Rectangle(
    bounds=[bbox1_min[::-1], bbox1_max[::-1]],  # Reverse (lat, lon) order
    color="blue",
    weight=2,
    fill=True,
    fill_opacity=0.3,
    popup="Sentinel 2"
).add_to(m)

# Add second bounding box (df_ign)
folium.Rectangle(
    bounds=[bbox2_min[::-1], bbox2_max[::-1]],  # Reverse (lat, lon) order
    color="red",
    weight=2,
    fill=True,
    fill_opacity=0.3,
    popup="IGN "
).add_to(m)
m

In [ ]:
filterer = PcdFilter(df=df, df_target=df_ign)
filterer.set_bounding_box_from_target(margin=6)
df_filtered = filterer.filter_data()
pcd = filterer.concatenate_dataframes([df_filtered, df_ign])

In [7]:
handler = PointCloudHandler(pcd)
handler.to_open3d()
handler.save_point_cloud(filename="final_IGN_LiDAR_HD.ply")

Point cloud saved to final_IGN_LiDAR_HD.ply
